# Building a GitHub Agent with LangGraph and LangChain

This notebook guides you through building an intelligent agent that can interact with GitHub repositories. Using LangGraph, LangChain, and IBM's Granite LLM, you'll create an agent capable of performing various GitHub operations through natural language instructions.

## What You'll Learn
- How to set up API connections to GitHub
- How to serve and interact with an LLM (IBM Granite)
- How to create tools that interact with GitHub's API
- How to build a ReAct agent that can reason about GitHub tasks

## Prerequisites
- GitHub account with a repository you want the agent to access
- Either a local Ollama server or a Replicate API token
- Basic understanding of Python and LLMs

Let's start by insuring we are using Python 3.10 or 3.11

In [ ]:

import sys
assert sys.version_info >= (3, 10) and sys.version_info < (3, 12), "Use Python 3.10 or 3.11 to run this notebook."

and Installing some dependencies:

In [ ]:
! pip install langgraph
! pip install --upgrade --quiet pygithub langchain-community replicate
! pip install git+https://github.com/ibm-granite-community/utils \
  langchain_ollama

## Connecting to GitHub

For your agent to interact with GitHub, it needs proper authentication. We'll use GitHub Apps for this purpose, which provides a secure way to access repositories.

### Setting Up Your GitHub App

1. Follow the instructions [here](https://docs.github.com/en/apps/creating-github-apps/registering-a-github-app/registering-a-github-app) to create a GitHub App
2. Configure the following permissions for your app:
   - Repository contents: Read & Write
   - Issues: Read & Write
   - Pull requests: Read & Write
3. Generate a private key for your app
4. Install the app to your repository

After setting up your GitHub App, you'll need three pieces of information:
- Your repository name in the format `username/repo-name`
- The private key file (downloaded when you created the app)
- The GitHub App ID (found in your app's settings)

Let's set these as environment variables:

In [ ]:
import os
from getpass import getpass

# Setup GitHub credentials
os.environ["GITHUB_REPOSITORY"] = "your/githubrepo"
os.environ["GITHUB_APP_PRIVATE_KEY"] = "your/private_key.pem"
os.environ["GITHUB_APP_ID"] = "GithubAppID"

## Setting Up the LLM

Our agent needs an LLM to generate responses and make decisions. We'll use IBM's Granite model, which can be served in two ways:

1. **Local deployment via Ollama**: Faster and free, but requires local resources
2. **Cloud deployment via Replicate**: Accessible from anywhere, but requires an API token

The code below will try to use a local Ollama server first, and if that's not available, it will fall back to Replicate.

### Setting Up Ollama (Optional)
If you want to use Ollama:
1. Install Ollama from [https://ollama.ai/](https://ollama.ai/)
2. Run `ollama pull granite3.1-dense:8b` to download the model

### Setting Up Replicate
If you want to use Replicate:
1. Create an account at [https://replicate.com/](https://replicate.com/)
2. Generate an API token at [https://replicate.com/account/api-tokens](https://replicate.com/account/api-tokens)

Let's set up our LLM:

In [ ]:
import os
import requests
from langchain_ollama.llms import OllamaLLM
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var
from ibm_granite_community.notebook_utils import get_env_var

# Setup Replicate API token (you need to provide this)
replicate_api_token = getpass("Enter your Replicate API token: ")
os.environ["REPLICATE_API_TOKEN"] = replicate_api_token

try: # Look for a locally accessible Ollama server for the model
    response = requests.get(os.getenv("OLLAMA_HOST", "http://127.0.0.1:11434"))
    llm = OllamaLLM(
        model="granite3.1-dense:8b",
        num_ctx=65536, # 64K context window
    )
# Initialize Replicate
except Exception:
  llm = Replicate(
    model ="ibm-granite/granite-3.1-8b-instruct",
      replicate_api_token = get_env_var('REPLICATE_API_TOKEN'),
      model_kwargs={
          "max_tokens": 2000, # Set the maximum number of tokens to generate as output.
          "min_tokens": 200, # Set the minimum number of tokens to generate as output.
          "temperature": 0.75,
          "presence_penalty": 0,
          "frequency_penalty": 0,
    }
)

## Creating the GitHub Toolkit

Now that we have our GitHub credentials and LLM set up, let's create the GitHub toolkit. This toolkit provides a set of tools that our agent can use to interact with GitHub.

LangChain provides a convenient `GitHubToolkit` class that wraps around the GitHub API. Let's initialize it:

In [ ]:
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper

github = GitHubAPIWrapper()
toolkit = GitHubToolkit.from_github_api_wrapper(github)

## Exploring Available GitHub Tools

Let's explore the tools that are available in our GitHub toolkit. Each tool represents a specific action our agent can take on GitHub, such as creating an issue, commenting on a pull request, or reading file contents.

This step helps us understand what our agent is capable of doing:

In [ ]:
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name)

## Testing the LLM

Before building our agent, it's a good idea to test the LLM to make sure it's working properly. This simple test ensures that we can connect to either Ollama or Replicate and that the model responds as expected:

In [ ]:
# Test the LLM to make sure it's working
llm.invoke("Hello, what can you help me with today?")

## Displaying Available GitHub Tools with Descriptions

Now let's get a better understanding of each tool by displaying not just their names, but also their descriptions. This will help us understand what each tool does and how our agent might use them:

In [ ]:
tools = toolkit.get_tools()
print("Available tools:")
for tool in tools:
    print(f"- {tool.name}")


## Building the ReAct Agent

Now we're ready to build our GitHub agent! We'll use the ReAct (Reasoning and Acting) framework, which allows our agent to:

1. Think about what it should do (Reasoning)
2. Take an action using one of the GitHub tools (Acting)
3. Observe the result of that action
4. Reason about what to do next based on those observations

This approach allows the agent to break down complex GitHub tasks into a series of steps.

We'll start by defining a prompt template that instructs the LLM on how to use the ReAct framework:

In [ ]:
from langchain.prompts import PromptTemplate

template = """You are a helpful GitHub assistant who can perform various actions on GitHub repositories.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, must be one of [{tool_names}]
Action Input: input passed to tool
Observation: action result
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""

# Create the prompt template
prompt = PromptTemplate(
    template=template,
    input_variables=["input", "agent_scratchpad"],
    partial_variables={
        "tools": lambda x: format_tool_string(tools),
        "tool_names": lambda x: ", ".join([tool.name for tool in tools]),
    },
)

# Helper function to format tools
def format_tool_string(tools):
    return "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

# Create a memory to maintain conversation context
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Instantiate the ReAct agent using LangChain's create_react_agent
agent = create_react_agent(
    llm,
    tools,
    prompt,
)

# Create an agent executor with the ReAct agent
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
)


## Creating a Helper Function to Run the Agent

Now that our agent is set up, let's create a helper function that makes it easy to run the agent with different queries:

In [ ]:
def run_github_agent(query):
    """Run the GitHub agent with a user query."""
    result = agent_executor.invoke({"input": query})
    return result

## Testing the GitHub Agent

Now that our agent is fully set up, let's test it with a simple query. This example asks the agent to count the number of issues in the repository:

In [ ]:
result = run_github_agent("how many issues are there")
print(result["output"])

## Example Use Cases

Here are some example queries you can try with your GitHub agent:

```python
# Get information about repositories
run_github_agent("List all the files in the repository")
run_github_agent("What are the most recent commits?")

# Work with issues
run_github_agent("Create a new issue titled 'Update documentation' with description 'We need to update the README file'")
run_github_agent("List all open issues")
run_github_agent("Close issue #1")

# Work with pull requests
run_github_agent("List all open pull requests")
run_github_agent("Comment on pull request #2 saying 'Looks good, approved!'")

# Work with file contents
run_github_agent("What's in the README.md file?")
run_github_agent("Create a new file called 'hello.txt' with content 'Hello, world!'")
```

Feel free to experiment with different queries based on your repository's contents and what you want to accomplish!

## Conclusion

Congratulations! You've built a functional GitHub agent that can interact with your repository using natural language instructions. This agent demonstrates the power of combining LLMs with API tools to create useful AI assistants.

### What You've Learned
- How to set up GitHub App authentication
- How to connect to and utilize an LLM (IBM Granite)
- How to create a ReAct agent that can reason about and execute GitHub operations
- How to use LangChain and LangGraph to create an AI agent with tool-using capabilities

### Next Steps
- Try more complex queries and see how the agent handles them
- Add more specialized tools for specific GitHub operations
- Integrate this agent with other services like Slack or Discord
- Refine the agent's prompt to make it better at specific tasks